**Proyecto:** INE Incidentes de Transito.

**Objetivo:** Ingesta de Hechos de Transito.

**Dependencias / Notebooks:** ingest_fallecidos_lesionados

**Ambiente de Ejecución:** Databricks, Azure, Desarrollo

---
| Fecha | Desarrollador | Descripcion |
| :--- | :--- | :--- |
| 06/NOV/20025 | Ruben Coloma | Emision Inicial |

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

**LEER LOS DATOS DEL CONTENEDOR**

In [0]:
# Constantes
str_container = "bronze"
str_storage_name = "stinetraficodeveastus2"
str_storage_path = f"abfss://{str_container}@{str_storage_name}.dfs.core.windows.net/"
str_file_transito = "08BaseDatosHechosTransitoPNC2023.csv"
str_ruta_transito = str_storage_path + str_file_transito

In [0]:
# Cargar cargar datos
df_transito_raw = spark.read.option('header', True)\
                        .option('inferSchema', True)\
                        .csv(str_ruta_transito)

In [0]:
# Quitar caracteres especiales nombres de columnas
df_transito_raw = df_transito_raw.withColumnRenamed("núm_corre", "num_corre")
df_transito_raw = df_transito_raw.withColumnRenamed("año_ocu", "anio_ocu")
df_transito_raw = df_transito_raw.withColumnRenamed("día_ocu", "dia_ocu")
df_transito_raw = df_transito_raw.withColumnRenamed("día_sem_ocu", "dia_sem_ocu")
df_transito_raw = df_transito_raw.withColumnRenamed("g_edad_80ymás", "g_edad_80ymas")
df_transito_raw = df_transito_raw.withColumnRenamed("g_edad_60ymás", "g_edad_60ymas")

**JOINS PARA CONVERTIR LOS CATALOGOS EN VALORES NUMERICOS**

In [0]:
# Cargar las tablas dimensionales
df_dim_hora = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_hora")
df_dim_g_hora_5 = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_g_hora_5")
df_dim_mes = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_mes_ocu")
df_dim_dia_sem_ocu = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_dia_sem_ocu")
df_dim_depto = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_depto_ocu")
df_dim_mupio = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_mupio_ocu")
df_dim_tipo_veh = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_tipo_veh")
df_dim_marca_veh = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_marca_veh")
df_dim_color_veh = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_color_veh")
df_dim_modelo_veh = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_modelo_veh")
df_dim_g_modelo_veh = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_g_modelo_veh")
df_dim_tipo_eve = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_tipo_eve")

In [0]:
# Realizar los joins
df_transito_ingest = df_transito_raw \
    .join(df_dim_hora, df_transito_raw.g_hora == df_dim_hora.g_hora, "left") \
    .join(df_dim_g_hora_5, df_transito_raw.g_hora_5 == df_dim_g_hora_5.g_hora_5, "left") \
    .join(df_dim_mes, df_transito_raw.mes_ocu == df_dim_mes.mes_ocur, "left") \
    .join(df_dim_dia_sem_ocu, df_transito_raw.dia_sem_ocu == df_dim_dia_sem_ocu.mes_ocur, "left") \
    .join(df_dim_depto, df_transito_raw.depto_ocu == df_dim_depto.depto_ocu, "left") \
    .join(df_dim_mupio, df_transito_raw.mupio_ocu == df_dim_mupio.mupio_ocu, "left") \
    .join(df_dim_tipo_veh, df_transito_raw.tipo_veh == df_dim_tipo_veh.tipo_veh, "left") \
    .join(df_dim_marca_veh, df_transito_raw.marca_veh == df_dim_marca_veh.marca_veh, "left") \
    .join(df_dim_color_veh, df_transito_raw.color_veh == df_dim_color_veh.color_veh, "left") \
    .join(df_dim_modelo_veh, df_transito_raw.modelo_veh == df_dim_modelo_veh.modelo_veh, "left") \
    .join(df_dim_g_modelo_veh, df_transito_raw.g_modelo_veh == df_dim_g_modelo_veh.g_modelo_veh, "left") \
    .join(df_dim_tipo_eve, df_transito_raw.tipo_eve == df_dim_tipo_eve.tipo_eve, "left") \

# Seleccionar las columnas finales (IDs numéricos en lugar de cadenas)
df_transito_ingest = df_transito_ingest.select(
    "num_corre", 
    "anio_ocu", 
    "dia_ocu", 
    "hora_ocu",
    df_dim_hora.id_g_hora.alias("id_g_hora"),
    df_dim_g_hora_5.id_g_hora_5.alias("id_g_hora_5"),
    df_dim_mes.id_mes_ocurrio.alias("id_mes_ocurrio"),
    df_dim_dia_sem_ocu.id_dia_sem_ocu.alias("id_dia_sem_ocu"),
    df_dim_depto.id_depto_ocu.alias("id_depto_ocu"),
    df_dim_mupio.id_mupio_ocu.alias("id_mupio_ocu"),
    "zona_ocu",
    df_dim_tipo_veh.id_tipo_veh.alias("id_tipo_veh"),
    df_dim_marca_veh.id_marca_veh.alias("id_marca_veh"),
    df_dim_color_veh.id_color_veh.alias("id_color_veh"),
    df_dim_modelo_veh.id_modelo_veh.alias("id_modelo_veh"),
    df_dim_g_modelo_veh.id_g_modelo_veh.alias("id_g_modelo_veh"),
    df_dim_tipo_eve.id_tipo_eve.alias("id_tipo_eve")
)

**GUARDAR LOS DATOS PARA TRASFORMACIONES**

In [0]:
df_transito_ingest.write.mode("overwrite").saveAsTable("cat_ine_trafico_dev.bronze_ine_trafico_dev.ine_aux_hechos_transito")